# Summary:
- Only the abstracts are used to get the vectors
- Experiment results:

    - Validation set accuracy =  0.7833821682712432



In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [5]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [6]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 1024), (14333,), (4778, 1024), (4778,), 19111)

In [7]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [8]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],32,32) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],32,32) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [9]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [10]:
model = model(input_shape = (32,32))

In [11]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 768)            369408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 768)            3072      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 768)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 768)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 512)            2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
__________

In [12]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [13]:
model.fit(X_train, Y_train, batch_size = 64, epochs=200)

Epoch 1/200
14333/14333 [==============================] - 14s 962us/step - loss: 0.5350 - acc: 0.7391
Epoch 2/200
14333/14333 [==============================] - 8s 585us/step - loss: 0.4768 - acc: 0.7819
Epoch 3/200
14333/14333 [==============================] - 8s 583us/step - loss: 0.4565 - acc: 0.7931
Epoch 4/200
14333/14333 [==============================] - 8s 591us/step - loss: 0.4465 - acc: 0.7965
Epoch 5/200
14333/14333 [==============================] - 8s 592us/step - loss: 0.4401 - acc: 0.7999
Epoch 6/200
14333/14333 [==============================] - 8s 593us/step - loss: 0.4318 - acc: 0.8063
Epoch 7/200
14333/14333 [==============================] - 8s 592us/step - loss: 0.4267 - acc: 0.8091
Epoch 8/200
14333/14333 [==============================] - 8s 588us/step - loss: 0.4213 - acc: 0.8132
Epoch 9/200
14333/14333 [==============================] - 8s 591us/step - loss: 0.4161 - acc: 0.8129
Epoch 10/200
14333/14333 [==============================] - 9s 596us/step - loss:

14333/14333 [==============================] - 8s 576us/step - loss: 0.3031 - acc: 0.8691
Epoch 160/200
14333/14333 [==============================] - 8s 578us/step - loss: 0.2986 - acc: 0.8738
Epoch 161/200
14333/14333 [==============================] - 8s 577us/step - loss: 0.3027 - acc: 0.8696
Epoch 162/200
14333/14333 [==============================] - 8s 576us/step - loss: 0.2998 - acc: 0.8714
Epoch 163/200
14333/14333 [==============================] - 8s 578us/step - loss: 0.2999 - acc: 0.8715
Epoch 164/200
14333/14333 [==============================] - 8s 577us/step - loss: 0.3010 - acc: 0.8739
Epoch 165/200
14333/14333 [==============================] - 8s 591us/step - loss: 0.3056 - acc: 0.8719
Epoch 166/200
14333/14333 [==============================] - 8s 589us/step - loss: 0.3010 - acc: 0.8703
Epoch 167/200
14333/14333 [==============================] - 8s 592us/step - loss: 0.3003 - acc: 0.8734
Epoch 168/200
14333/14333 [==============================] - 9s 593us/step - l

In [14]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4778/4778 [==============================] - 2s 319us/step
Validation set accuracy =  0.7833821682712432
